In [5]:
import findspark

In [6]:
findspark.init()

In [8]:
import pyspark

In [11]:
sc = pyspark.SparkContext()

In [13]:
titanic_train = sc.textFile("/home/mpean/Documents/formations/cursus-big-data/data-formation-hadoop/data/titanic_train.csv")

In [15]:
titanic_train.take(5)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,-,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,-,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']

In [18]:
titanic_train.count()

892

suppression de la ligne contenant le nom des champs

In [21]:
titanic_train_data = titanic_train.filter(lambda ligne : 'PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked' not in ligne)

In [22]:
titanic_train_data.take(5)

['1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,-,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,-,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S',
 '5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,-,S']

In [24]:
titanic_train_data.count()

891

In [27]:
ttd = titanic_train_data.map(lambda ligne : ligne.split('"'))
ttd.take(5)

[['1,0,3,', 'Braund, Mr. Owen Harris', ',male,22,1,0,A/5 21171,7.25,-,S'],
 ['2,1,1,',
  'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  ',female,38,1,0,PC 17599,71.2833,C85,C'],
 ['3,1,3,',
  'Heikkinen, Miss. Laina',
  ',female,26,0,0,STON/O2. 3101282,7.925,-,S'],
 ['4,1,1,',
  'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
  ',female,35,1,0,113803,53.1,C123,S'],
 ['5,0,3,', 'Allen, Mr. William Henry', ',male,35,0,0,373450,8.05,-,S']]


on supprime le nom de la personne. Le [:-1] permet de supprimer une virgule en trop. Les champs restants sont : 'PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked'

In [28]:
ttd1 = ttd.map(lambda s: s[0][:-1] + s[-1])
ttd1.take(15)

['1,0,3,male,22,1,0,A/5 21171,7.25,-,S',
 '2,1,1,female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,female,26,0,0,STON/O2. 3101282,7.925,-,S',
 '4,1,1,female,35,1,0,113803,53.1,C123,S',
 '5,0,3,male,35,0,0,373450,8.05,-,S',
 '6,0,3,male,-,0,0,330877,8.4583,-,Q',
 '7,0,1,male,54,0,0,17463,51.8625,E46,S',
 '8,0,3,male,2,3,1,349909,21.075,-,S',
 '9,1,3,female,27,0,2,347742,11.1333,-,S',
 '10,1,2,female,14,1,0,237736,30.0708,-,C',
 '11,1,3,female,4,1,1,PP 9549,16.7,G6,S',
 '12,1,1,female,58,0,0,113783,26.55,C103,S',
 '13,0,3,male,20,0,0,A/5. 2151,8.05,-,S',
 '14,0,3,male,39,1,5,347082,31.275,-,S',
 '15,0,3,female,14,0,0,350406,7.8542,-,S']

nombre de survivants

In [30]:
survivants = ttd1.map(lambda ligne : ligne.split(',')).filter(lambda person : int(person[1]))
survivants.count()

342

nombre d'hommes survivants

In [32]:
survivants_hommes = survivants.filter(lambda person : person[3] == 'male')
survivants_hommes.count()

109

nombre de femmes survivantes

In [34]:
survivants_femmes = survivants.filter(lambda person : person[3] == 'female')
survivants_femmes.count()

233

nombre de survivants hommes et femmes

In [36]:
survivants_genre = survivants.map(lambda person : (person[3], int(1))).reduceByKey(lambda a,b : a+b)
survivants_genre.take(5)

[('female', 233), ('male', 109)]

classement des survivants pas classe

In [38]:
survivants_classe = survivants.map(lambda person : (person[2], 1)) \
    .reduceByKey(lambda a,b : a+b) \
    .sortBy(lambda x : x[1], False)
survivants_classe.take(5)

[('1', 136), ('3', 119), ('2', 87)]